In [42]:
### Stream Ciphers
## A stream cipher is a symmetric key cipher where plaintext digits are combined 
## with a pseudorandom cipher digit stream (keystream). In a stream cipher, 
## each plaintext digit is encrypted one at a time with the corresponding digit 
## of the keystream, to give a digit of the ciphertext stream. Since encryption 
## of each digit is dependent on the current state of the cipher, it is also 
## known as state cipher. In practice, a digit is typically a bit and the combining 
## operation is an exclusive-or (XOR).

### Linear congruential generator
## A linear congruential generator (LCG) is an algorithm that yields a sequence of 
## pseudo-randomized numbers calculated with a discontinuous piecewise linear equation. 
## The method represents one of the oldest and best-known pseudorandom number generator algorithms. 
## The theory behind them is relatively easy to understand, and they are easily implemented 
## and fast, especially on computer hardware which can provide modular arithmetic by storage-bit truncation.

In [43]:

import random


class KeyStream:

  def __init__(self, key=1):
    self.next = key

  def rand(self):
    self.next = (11035152245*self.next + 12345) % 2**31
    return self.next

  def get_key_byte(self):
    return (self.rand()//2**23) % 256

def encrypt(key, message):
  return bytes([message[i] ^ key.get_key_byte() for i in range(len(message))])

def transmit(cipher, likely):
  b = []
  for c in cipher:
    if random.randrange(0, likely) == 0:
      c = c ^ 2**random.randrange(0, 8)
    b.append(c)
  return bytes(b)

def modification(cipher):
  mod = [0]*len(cipher)
  mod[10] = ord(' ') ^ ord('1')
  mod[11] = ord(' ') ^ ord('0')
  mod[12] = ord('1') ^ ord('0')
  return bytes([mod[i] ^ cipher[i] for i in range(len(cipher))])

def get_key(message, cipher):
  return bytes([message[i] ^ cipher[i] for i in range(len(cipher))])

def crack(key_stream, cipher):
  length = min(len(key_stream), len(cipher))
  return bytes([key_stream[i] ^ cipher[i] for i in range(length)])

def brute_force(plain, cipher):
  for k in range(2**31):
    bf_key = KeyStream(k)
    for i in range(len(plain)):
      xor_value = plain[i] ^ cipher[i]
      if xor_value != bf_key.get_key_byte():
        break
    else:
      return k
  return False


key = KeyStream(10)
#for i in range(10):
#  print(key.get_key_byte())
message = "Hello, World! I am here to declare that I will take over the universe and become the supreme emperor.".encode()
print(message)
cipher = encrypt(key, message)
print(cipher)


cipher = transmit(cipher, 5)


key = KeyStream(10)
message = encrypt(key, cipher)
print(message)

b'Hello, World! I am here to declare that I will take over the universe and become the supreme emperor.'
b"*\xd66\t\x8f\x15{\x1f4\xcf\x12wR3+\xaf\xa3MN\x9e\t\xab\xad\r\xbec}\x81 \xdc8\xa9\x8e\xdf\xe9\x0f/PV\x85\xf9\xd5\xa0\xc6\xe6[,\xf3\xfd'\xae$p\x1b\\H\xa8\x97\xce\rk\xd2\xaa\xd5W\xbb\x90\xd4\x95\x0f\xe9\x92\x8aR%\xe7m\xccc,\x7f\xe2\x85\x02D\x01\xe7(w(9\xa2\x15dX\x95\xba\xe8\xf1b\x00"
b'hello, Worle! I am here t\xef dec|are\x00that I(wi|l tike over\x00dhe univursd\x00an` beCOme the supreme e-peror.'


In [44]:
# This is Alice
key = KeyStream(10)
message = "Send Bob:   10$".encode()
print(message)
cipher = encrypt(key, message)
print(cipher)

# This is Bob
cipher = modification(cipher)

# This is bank
key = KeyStream(10)
message = encrypt(key, cipher)
print(message)

b'Send Bob:   10$'
b'1\xd64\x01\xc0{4*a\x9d^3B#F'
b'Send Bob: 1000$'


In [45]:
# Eve goes to Alice
eves_message = "This is Eve's most valued secrets of all her life.".encode()

# This is Alice alone
key = KeyStream(10)
message = eves_message
print(message)
cipher = encrypt(key, message)
print(cipher)

# This is Eve (alone) all evil
eves_key_stream = get_key(eves_message, cipher)

# This is Bob
key = KeyStream(10)
message = encrypt(key, cipher)
print(message)

# Alice again
message = "Hi Bob, let's meet a plan our world domination.".encode()
key = KeyStream(10)
cipher = encrypt(key, message)
print(cipher)

# Bob again
key = KeyStream(10)
message = encrypt(key, cipher)
print(message)

# Eve again (more evil than ever)
print(" THIS IS EVE ")
print(crack(eves_key_stream, cipher))

b"This is Eve's most valued secrets of all her life."
b'6\xdb3\x16\xc0P(h\x1e\xcb\x1b4\x003\x0f\xe0\xb1TN\x80\r\xb5\xbdH\xae,.\x80&\xcd1\xbc\x8f\x9a\xa6\x1dgPN\xc9\x90\x9d\xb2\xdd\xaa[e\xe1\xf9b'
b"This is Eve's most valued secrets of all her life."
b'*\xdaz\'\x8f[wh7\xd8\n4\x003\x0f\xea\xa7TN\x97L\xa9\xa4L\xa4,2\x907\x9f#\xa7\x8e\xd6\xad[#^O\xcc\xde\x94\xa3\xc6\xe5Y"'
b"Hi Bob, let's meet a plan our world domination."
 THIS IS EVE 
b"Hi Bob, let's meet a plan our world domination."


In [46]:
## What is entropy in cryptography?
## In cryptography, entropy refers to the randomness collected by a system for use in algorithms
## that require random data. A lack of good entropy can leave a cryptosystem vulnerable
## and unable to encrypt data securely.

# Brute force of our Stream Cipher - More than one weakness revelead

# This is Alice
secret_key = random.randrange(0, 2**20)
print(secret_key)
key = KeyStream(secret_key)
header = "MESSAGE: "
message = header + "My secret message to Bob"
message = message.encode()
print(message)
cipher = encrypt(key, message)
print(cipher)

# This is Bob
key = KeyStream(secret_key)
message = encrypt(key, cipher)
print(message)

# This is Eve
bf_key = brute_force(header.encode(), cipher)
print("Eve's brute force key:", bf_key)
key = KeyStream(bf_key)
message = encrypt(key, cipher)
print(message)


86565
b'MESSAGE: My secret message to Bob'
b'\xeb:\x0f@\xa5\xe9\xdc"j\x01\x12\x90\x17\x07\r\x06t\xc9\x13}\xa6I\xd5\xef\xf6\xf8\x7f\xd7\x8f,\x8b:\xa5'
b'MESSAGE: My secret message to Bob'
Eve's brute force key: 86565
b'MESSAGE: My secret message to Bob'
